In [1]:
import pandas as pd
import numpy as np

import math
import scipy.optimize as opt
import random
random.seed(109)


In [2]:
# Data: Close
db = pd.read_excel('Data.xlsx', index_col = "Date")

# MSD

In [3]:
db.index = pd.to_datetime(db.index)
MSD_df = db['2006':'2013']
MSD_df = (MSD_df-MSD_df.mean())/MSD_df.std() #change rolling
MSD_df = MSD_df.dropna(axis = 1) #drop nan

In [4]:
target = {}
for i in range(0,len(MSD_df.columns)-1):
    for j in range(i+1,len(MSD_df.columns)):
        tmp = ((MSD_df.iloc[:,i] - MSD_df.iloc[:,j])**2).sum()
        target[str(MSD_df.columns[i])+','+str(MSD_df.columns[j])] = tmp

In [5]:
target_df = pd.DataFrame.from_dict(target, orient='index')
target_df = target_df.sort_values(by = 0)[0:10]
target_df

,0
"DIS,TRV",106.019533
"IBM,MCD",174.553811
"AAPL,IBM",183.653680
"CVX,KO",205.031258
"IBM,KO",225.295065
"DIS,HD",264.348677
"HD,TRV",282.786437
"PEP,PG",298.596370
"DIS,MMM",300.548727
"GOOGL,TRV",304.718140


In [6]:
pair1_df_train = db.loc["2006":"2013",target_df.index[3].split(",")].pct_change(1).dropna()
pair1_df_test = db.loc["2014":"2014",target_df.index[3].split(",")].pct_change(1).dropna()
pair1_df_train

,CVX,KO
Date,,
2006-01-04,-0.002877,-0.001956
2006-01-05,-0.012222,0.004900
2006-01-06,0.018216,0.009508
2006-01-09,-0.005063,0.004830
2006-01-10,0.004919,0.000961
...,...,...
2013-12-24,0.005782,0.000747
2013-12-26,0.010525,0.007465
2013-12-27,0.003365,0.004199


In [7]:
# ytlist = pair1_df_train.iloc[:,0] - pair1_df_train.iloc[:,1]

In [8]:
from scipy.stats import t

# ytlist = [] #用來收集過去的yt們，算z_td、s_z時會用到

# #Table1 參數
# fi_1_0 = 0.1046
# fi_1_1 = 0.3886
# fi_2_0 = 0.0961
# fi_2_1 = -0.2403
# alpha_1_0 = 0.1639
# alpha_1_1 = 0.2132
# beta_1_1 = 0.6631
# alpha_2_0 = -0.1035
# alpha_2_1 = -0.1063
# beta_2_1 = -0.1751
# v = 7
# gamma = 5.5563
# c_1 = -0.3569
# c_2 = 0.3136
# d = 1
# z_td = yt[len(yt)-d:-1] #t-d
# s_z = std(z_td)

# Fdis = (1+np.exp(-gamma*(z_td - c_1)(z_td - c_2))/s_z)**(-1)
# h_1_t = alpha_1_0 + alpha_1_1*a_t1**2 + beta_1_1*h_t1
# h_2_t = alpha_2_0 + alpha_2_1*a_t1**2 + beta_2_1*h_t1
# miu_1 = fi_1_0 + fi_1_1*y_t1
# miu_2 = fi_2_0 + fi_2_1*y_t1
# h_t = h_1_t + Fdis*h_2_t
# epsilon = t.rvs(v, size=1) #t-distribution
# a_t = h_t**0.5*epsilon #error term
# y_t = miu_1 + Fdis*miu_2 + a_t #ST-GARCH


In [9]:
z_td = pair1_df_train.iloc[0:30,0]-pair1_df_train.iloc[0:30,1]
z_td

Date
2006-01-04   -0.000921
2006-01-05   -0.017122
2006-01-06    0.008709
2006-01-09   -0.009893
2006-01-10    0.003958
2006-01-11    0.009648
2006-01-12    0.006355
2006-01-13    0.010985
2006-01-17    0.031629
2006-01-18   -0.010401
2006-01-19    0.008799
2006-01-20    0.012731
2006-01-23    0.007607
2006-01-24   -0.012449
2006-01-25   -0.018600
2006-01-26   -0.025241
2006-01-27   -0.009156
2006-01-30    0.011370
2006-01-31   -0.013689
2006-02-01   -0.014352
2006-02-02    0.003269
2006-02-03   -0.008773
2006-02-06    0.021837
2006-02-07   -0.020893
2006-02-08   -0.002277
2006-02-09   -0.011523
2006-02-10   -0.008999
2006-02-13    0.010381
2006-02-14   -0.030623
2006-02-15    0.004307
dtype: float64

In [10]:
# fi_1_0, fi_1_1, fi_2_0, fi_2_1, alpha_1_0, alpha_1_1, beta_1_1, alpha_2_0, alpha_2_1, beta_2_1, v, gamma, c_1, c_2 = np.array([0.1046, 0.3886, 0.0961, -0.2403, 0.1639, 0.2132, 0.6631, -0.1035, -0.1063, -0.1751, 7.1122, 5.5563, -0.3569, 0.3136])
# tmp = 30
# z_td = pair1_df_train.iloc[0:tmp,0] - pair1_df_train.iloc[0:tmp,1]
# s_z = z_td.std()   
# Fdis = (1+np.exp(-gamma*(z_td - c_1)*(z_td - c_2)/s_z))**(-1)
# Fdis

In [11]:
def fun_MLE(x):
    fi_1_0, fi_1_1, fi_2_0, fi_2_1, alpha_1_0, alpha_1_1, beta_1_1, alpha_2_0, alpha_2_1, beta_2_1, gamma, c_1, c_2= x
    # Define h_t1
    tmp = 30
    target = 0
    z_td = pair1_df_train.iloc[0:tmp,0]-pair1_df_train.iloc[0:tmp,1]
    z_td = z_td.to_numpy()
    s_z = z_td.std()
    Fdis = (1+np.exp(-gamma*(z_td[-1] - c_1)*(z_td[-1] - c_2)/s_z))**(-1)
    h_1_t = pair1_df_train.iloc[0:tmp,0].std()
    h_2_t = pair1_df_train.iloc[0:tmp,1].std()
    h_t1 = h_1_t + Fdis*h_2_t
    epsilon = random.gauss(mu = 0, sigma = 1)
    a_t1 = h_t1**0.5*epsilon #error term


    y_t1 = pair1_df_train.iloc[tmp-1,0] - pair1_df_train.iloc[tmp-1,1]
    # Define sigma, real_y_t, y_t
    for i in range(tmp,len(pair1_df_train)):
        z_td = pair1_df_train.iloc[0:i,0]-pair1_df_train.iloc[0:i,1]
        z_td = z_td.to_numpy()
        s_z = z_td.std()
        real_y_t = pair1_df_train.iloc[i,0] - pair1_df_train.iloc[i,1]
        # real_y_t = real_y_t.to_numpy()
        h_1_t = alpha_1_0 + alpha_1_1*a_t1**2 + beta_1_1*h_t1
        h_2_t = alpha_2_0 + alpha_2_1*a_t1**2 + beta_2_1*h_t1
        miu_1 = fi_1_0 + fi_1_1*y_t1
        miu_2 = fi_2_0 + fi_2_1*y_t1
        h_t = h_1_t + Fdis*h_2_t
        epsilon = random.gauss(mu = 0, sigma = 1)
        a_t = h_t**0.5*epsilon #error term
        y_t = miu_1 + Fdis*miu_2 + a_t #ST-GARCH

        h_t1 = h_t
        a_t1 = a_t
        y_t1 = real_y_t

        sigma = h_t
        target += np.log((2*np.pi*sigma**2)**(-0.5)*np.exp(-(real_y_t-y_t)**2/(2*sigma**2))) #MLE
        # print(target)
    target = -target
    
    return target

In [12]:
#MLE
tmp = 1e-9*4
initial = np.array([0.1068, 0.3838, 0.0940, -0.2366, 0.1649, 0.2062, 0.6622, -0.1045, -0.0981, -0.1827, 4.9262, -0.3516, 0.3068]) 
b1 = ((pair1_df_train.iloc[0:30,0] - pair1_df_train.iloc[0:30,1]).std())**2*10000
b2 = 1
b3 = 1.1
constraints = ( {'type':'ineq','fun':lambda x: x[4]+tmp},
                {'type':'ineq','fun':lambda x: x[5]+tmp},
                {'type':'ineq','fun':lambda x: x[6]+tmp},
                {'type':'ineq','fun':lambda x: x[5]+x[8]+tmp},
                {'type':'ineq','fun':lambda x: x[6]+x[9]+tmp},
                {'type':'ineq','fun':lambda x:-x[5]-0.5*x[8]-x[6]-0.5*x[9]+1+tmp},
                {'type':'ineq','fun':lambda x:-x[4]+b1+tmp},     #b1
                {'type':'ineq','fun':lambda x:-x[6]+b2+tmp},     #b2 = 1
                {'type':'ineq','fun':lambda x:-x[5]-x[6]+b3+tmp},#b3 = 1.1
                {'type':'ineq','fun':lambda x: x[12]-x[11]+tmp} 
                  )
bound = (-0.9,0.9)
# bounds = tuple(bound for asset in range(len(initial)))
bounds = ((-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-0.9, 0.9),
 (-5, 5),
 (-0.9, 0.9),
 (-0.9, 0.9))
minimum = opt.minimize(fun_MLE, initial, constraints = constraints, method = 'trust-constr', bounds = bounds) #bounds: 0.9
minimum

/home/akilin/anaconda3/envs/gamma/lib/python3.7/site-packages/scipy/optimize/_hessian_update_strategy.py:186: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 1874
      cg_stop_cond: 2
            constr: [array([0.19563799]), array([0.33571863]), array([0.29609246]), array([0.37727849]), array([0.21160178]), array([0.38965433]), array([1.88653496]), array([0.70390755]), array([0.46818892]), array([1.00956854]), array([ 0.14130425,  0.42637775,  0.14471445, -0.05919238,  0.19563799,
        0.33571863,  0.29609246,  0.34740389,  0.04155986, -0.08449068,
        4.57677558, -0.4611008 ,  0.54846773])]
       constr_nfev: [6524, 6524, 6524, 6524, 6524, 6524, 6524, 6524, 6524, 6524, 0]
       constr_nhev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
       constr_njev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 2779.21945977211
               fun: 2587.425171620171
              grad: array([ 1.10856947e+10,  6.08858242e+09,  1.99151053e+10, -1.13786700e+10,
        1.12649751e+10, -1.14701491e

In [13]:
minimum.x

array([ 0.14130425,  0.42637775,  0.14471445, -0.05919238,  0.19563799,
        0.33571863,  0.29609246,  0.34740389,  0.04155986, -0.08449068,
        4.57677558, -0.4611008 ,  0.54846773])

In [14]:
np.savetxt(str(target_df.index[3].split(",")[0])+'_'+str(target_df.index[3].split(",")[1])+'.txt', minimum.x, delimiter=',' )

# DIS, TRV

In [15]:
minimum.x

array([ 0.14130425,  0.42637775,  0.14471445, -0.05919238,  0.19563799,
        0.33571863,  0.29609246,  0.34740389,  0.04155986, -0.08449068,
        4.57677558, -0.4611008 ,  0.54846773])

# Trade

In [16]:
df_return = db.pct_change()

In [17]:
global results

In [18]:
results = {'signals':[],
           'returns':[],
           'A_SoldPrices':[],
           'A_BoughtPrices':[],
           'B_SoldPrices':[],
           'B_BoughtPrices':[]
           }

In [19]:
# Input1 : Real return spread
# Input2 : Estimated return spread
# Output : Gain, Trading Signal

y1 = df_return.iloc[-1,0] 
y2 = df_return.iloc[-1,1]
p1 = db.iloc[-1:,0] #A Price
p2 = db.iloc[-1,1] #B Price

#First trade
i = 0

#signal
tmp0 = 0
tmp1 = 1
tmp2 = -1

#First trade
if i == 0:
    if (y1 > y2):
        signal = tmp1
        results['A_SoldPrices'].append(p1)
        results['B_BoughtPrices'].append(p2)
        i = 1
        tmp = 0
    
    elif (y1 < y2):  
        signal = tmp2
        results['A_BoughtPrices'].append(p1)
        results['B_SoldPrices'].append(p2) 
        i = 1
        tmp = 0
    else:
        signal = tmp0
        i = 0
        tmp = 1

elif i == 1:
    # After first trade
    if (y1 > y2) & (results['signals'][-1] == tmp1) :
        signal = tmp1
        results['A_SoldPrices'].append(y1)
        results['B_BoughtPrices'].append(y2)
        tmp = -np.log(results['A_SoldPrices'][-1]/results['A_BoughtPrices'][-1])+np.log(results['B_SoldPrices'][-1]/results['B_BoughtPrices'][-1])
  
    elif (y1 <y2) & (results['signals'][-1] == tmp2):  
        signal = tmp2
        results['A_BoughtPrices'].append(y1)
        results['B_SoldPrices'].append(y2)
        tmp = np.log(results['A_SoldPrices'][-1]/results['A_BoughtPrices'][-1])-np.log(results['B_SoldPrices'][-1]/results['B_BoughtPrices'][-1]) 

    else:
        signal = tmp0
        tmp = 0


results['signals'].append(signal)
results['returns'].append(tmp)